In [1]:
import sys
import os
import pickle
import csv

from io import BytesIO
import numpy as np

from rdflib import URIRef
from rdflib.namespace import ClosedNamespace


In [2]:
sys.path.append("/home/alireza/aida/tools/AIDA-Interchange-Format/python/aida_interchange")
from Bounding_Box import Bounding_Box
import aifutils


In [3]:
with open('../../../wsod/metadata/ont_300/300_joined_list.csv', 'r') as fin:
    all_labels = [item.strip() for item in fin]

In [4]:
with open('../../../../data/OpenImages/metadata-v4/class-names/class-descriptions.csv', 'r') as fin:
    mid2name = {item[0]: item[1] for item in csv.reader(fin)}

In [5]:
mid2name[all_labels[0]]

'Air force'

In [6]:
mid2term = {item: 'm_' + item.split('/')[-1] + '_' + mid2name[item].replace(' ', '_').replace('&', 'and') for item in all_labels}

In [7]:
with open('../../results/det_results_postproc_11.pkl', 'rb') as fin:
    det_results = pickle.load(fin)

In [8]:
with open('../../results/det_results_postproc_12.pkl', 'rb') as fin:
    det_results2 = pickle.load(fin)

In [9]:
VIS_ONTOLOGY = ClosedNamespace(
    uri=URIRef("http://www.columbia.edu/AIDA/DVMM/Ontologies/OpenImagesSubset01#"),
    terms=[mid2term[item] for item in all_labels]
)


## Loading the selected set

In [10]:
root_docs = ['IC0015MEL','IC0015N8B','IC0015R7B','IC0015S3J','IC0015S43','IC0015S4O','IC0015S7Y','IC0015S83','IC0015SWP','IC0015LGD']

In [11]:
root_to_leaf = {}
leaf_to_root = {}
flag = False
with open('../../../../data/seedling/parent_children_part2.tab', 'r') as fin:
    for line in fin:
        if not flag:
            flag = True
            continue
        row = line.split()
        if root_to_leaf.get(row[0]) == None:
            root_to_leaf[row[0]] = []
        root_to_leaf[row[0]].append(row[1])
        
        leaf_to_root[row[1]] = row[0]

In [12]:
leaf_docs = []
for item in root_docs:
    leaf_docs += root_to_leaf[item]

In [13]:
vid_ids = [item.split('.')[0] for item in leaf_docs if 'mp4' in item]

In [14]:
vid_ids

['IC0019N0N', 'IC0019N3V', 'IC0019NB0', 'IC0019NAH', 'HC0005KMS', 'HC0005KS5']

In [15]:
img_ids = [item for item in leaf_docs if 'jpg' in item]

In [16]:
len(img_ids)

92

In [17]:
vid_to_shot = {}

with open('../../../../data/seedling/seedling_keyframes/msb.txt', 'r') as fin:
    for line in fin:
        row = line.split()
        shotname = row[1]
        vidname = row[1].split('_')[0]
        
        if vid_to_shot.get(vidname) == None:
            vid_to_shot[vidname] = []
        vid_to_shot[vidname].append(shotname)

In [ ]:
g = aifutils.make_graph()

system_co = aifutils.make_system_with_uri(g, "http://www.columbia.edu/AIDA/DVMM/Systems/ObjectDetection/FasterRCNN-NASNet/COCO")
system_oi = aifutils.make_system_with_uri(g, "http://www.columbia.edu/AIDA/DVMM/Systems/ObjectDetection/FasterRCNN-InceptionResNet/OpenImages")
system_ws = aifutils.make_system_with_uri(g, "http://www.columbia.edu/AIDA/DVMM/Systems/ObjectDetection/MITWeaklySupervised-ResNet/OpenImages")



In [18]:
for imgid in img_ids:
    det = det_results[imgid]
    for ii in range(det['num_detections']):
        label = det['detection_classes'][ii]
        score = det['detection_scores'][ii]
        bbox = det['detection_boxes'][ii]
        model = det['detection_models'][ii]
        
        if score < 0.35:
            continue
        
        if model == 'coco':
            sys = system_co
        elif model == 'oi':
            sys = system_oi
        elif model == 'ws':
            sys = system_ws
        elif model == 'coco/J':
            sys = system_co
        elif model == 'oi/J':
            sys = system_oi
        elif model == 'ws/J':
            sys = system_ws

        labelrdf = VIS_ONTOLOGY.term(mid2term[label])
        
        entity = aifutils.make_entity(g, f"http://www.columbia.edu/AIDA/DVMM/Entities/ObjectDetection/RUN00002/{imgid}/{ii}", sys)

        type_assertion = aifutils.mark_type(g, f"http://www.columbia.edu/AIDA/DVMM/TypeAssertion/ObjectDetection/RUN00002/{imgid}/{ii}/1", 
                                            entity, labelrdf, sys, score)

        bb = Bounding_Box((bbox[0], bbox[1]), (bbox[2], bbox[3]))
        aifutils.mark_image_justification(g, [entity, type_assertion], imgid, bb, sys, score)
        



In [19]:
for vidid in vid_ids:
    for keyframeid in vid_to_shot[vidid]:
        imgid = keyframeid + '.png'
        det = det_results2[imgid]
        for ii in range(det['num_detections']):
            label = det['detection_classes'][ii]
            score = det['detection_scores'][ii]
            bbox = det['detection_boxes'][ii]
            model = det['detection_models'][ii]

            if score < 0.35:
                continue

            if model == 'coco':
                sys = system_co
            elif model == 'oi':
                sys = system_oi
            elif model == 'ws':
                sys = system_ws
            elif model == 'coco/J':
                sys = system_co
            elif model == 'oi/J':
                sys = system_oi
            elif model == 'ws/J':
                sys = system_ws

            labelrdf = VIS_ONTOLOGY.term(mid2term[label])

            entity = aifutils.make_entity(g, f"http://www.columbia.edu/AIDA/DVMM/Entities/ObjectDetection/RUN00002/{imgid}/{ii}", sys)

            type_assertion = aifutils.mark_type(g, f"http://www.columbia.edu/AIDA/DVMM/TypeAssertion/ObjectDetection/RUN00002/{imgid}/{ii}/1", 
                                                entity, labelrdf, sys, score)

            bb = Bounding_Box((bbox[0], bbox[1]), (bbox[2], bbox[3]))

            aifutils.mark_keyframe_video_justification(g, [entity, type_assertion], vidid, keyframeid, bb, sys, score)
        

In [20]:
with open('../../results/aida_output_13', 'w') as fout:
    serialization = BytesIO()
    # need .buffer because serialize will write bytes, not str
    g.serialize(destination=serialization, format='turtle')
    fout.write(serialization.getvalue().decode('utf-8'))


In [21]:
with open('../../temp/ontology_export_3.ttl', 'w') as fout:
    fout.write(
'''
@prefix : <http://www.columbia.edu/AIDA/DVMM/Ontologies/OpenImagesSubset01#> .
@prefix aidaDomainCommon:  <http://darpa.mil/ontologies/AidaDomainOntologiesCommon#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

: rdf:type owl:Ontology ;
    rdfs:label "DVMM Ontology"^^xsd:string ;
    rdfs:comment "Extracted from openimages. m_something shows freebase ID."^^xsd:string ;
    owl:versionInfo "Version 0.1"^^xsd:string .

'''
    )
    for label in all_labels:
        fout.write(f':{mid2term[label]} rdfs:subClassOf aidaDomainCommon:EntityType .\n')